In [1]:
import pandas as pd
import openpyxl

In [2]:
df = pd.read_excel("C:/Users/moise/OneDrive/Documentos/UPY/project/Estancia ENES/Scraper_and_mapping/Databases/dd11075.xlsx", engine="openpyxl", index_col=None)
df

,pk_anio,pk_mes,ngasto_d01,ngasto_d02,ngasto_d03,ngasto_d04,ngasto_d05,ngasto_d06,ngasto_d07,ngasto_d08,...,ngasto_d22,ngasto_d23,ngasto_d24,ngasto_d25,ngasto_d26,ngasto_d27,ngasto_d28,ngasto_d29,ngasto_d30,ngasto_d31
0,1978,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,4.637,0.798,47.340,24.650,47.370,70.020,39.550,40.730,NaN
1,1978,7,64.990,61.100,41.920,86.040,129.500,55.120,34.090,36.510,...,36.430,30.820,36.940,146.600,79.570,64.110,214.000,229.700,141.800,98.100
2,1978,8,101.000,55.310,45.970,51.100,40.230,39.950,89.110,32.810,...,62.040,91.900,75.350,88.190,71.240,68.560,79.550,98.730,149.000,262.800
3,1978,9,191.200,184.300,166.900,131.500,166.300,202.200,140.900,140.200,...,151.500,112.100,89.210,91.070,128.500,238.200,225.200,344.800,305.400,NaN
4,1978,10,288.100,285.800,293.500,319.100,288.100,311.300,268.100,265.800,...,44.780,45.530,55.890,95.000,78.630,47.340,41.160,34.720,30.630,28.250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
302,2008,8,306.420,360.599,322.065,275.142,290.290,310.127,359.703,272.226,...,741.959,736.340,589.667,546.679,494.684,568.640,816.844,1024.793,1212.564,1702.909
303,2008,9,1083.366,930.960,926.840,1013.482,1047.736,957.765,875.457,816.892,...,704.714,719.421,670.998,589.178,524.214,607.852,606.012,602.893,536.278,NaN
304,2008,10,493.125,444.765,420.390,396.390,377.712,364.961,353.561,333.433,...,297.624,289.839,291.072,282.953,274.706,265.670,267.753,267.438,259.588,247.295
305,2008,11,219.871,215.948,211.598,203.559,194.170,190.446,189.052,185.316,...,106.261,24.306,17.290,12.090,12.829,13.089,13.015,12.503,12.241,NaN


In [3]:
df.columns

Index(['pk_anio', 'pk_mes', 'ngasto_d01', 'ngasto_d02', 'ngasto_d03',
       'ngasto_d04', 'ngasto_d05', 'ngasto_d06', 'ngasto_d07', 'ngasto_d08',
       'ngasto_d09', 'ngasto_d10', 'ngasto_d11', 'ngasto_d12', 'ngasto_d13',
       'ngasto_d14', 'ngasto_d15', 'ngasto_d16', 'ngasto_d17', 'ngasto_d18',
       'ngasto_d19', 'ngasto_d20', 'ngasto_d21', 'ngasto_d22', 'ngasto_d23',
       'ngasto_d24', 'ngasto_d25', 'ngasto_d26', 'ngasto_d27', 'ngasto_d28',
       'ngasto_d29', 'ngasto_d30', 'ngasto_d31'],
      dtype='object')

In [4]:
# Si 'pk_anio' está como índice, lo convertimos en columna
if 'level_0' in df.columns:
    df = df.drop(columns=['level_0'])
df.reset_index(inplace=True)

# Lista de nombres de meses
nombres_meses = ["ene.", "feb.", "mar.", "abr.", "may.", "jun.", "jul.", "ago.", "sep.", "oct.", "nov.", "dic."]

# Transformar el DataFrame de formato ancho a largo
df_long = df.melt(id_vars=["pk_anio", "pk_mes"], var_name="dia", value_name="valor")

# Extraer solo los números de día (ngasto_d01 → 1), ignorando los NaN
df_long["dia"] = df_long["dia"].str.extract(r"(\d+)")

# **Manejamos los NaN antes de convertir a int**
df_long = df_long.dropna(subset=["dia"])
df_long["dia"] = df_long["dia"].astype(int)

# **Convertimos 'pk_mes' y 'pk_anio' a int**
df_long["pk_mes"] = df_long["pk_mes"].astype(int)
df_long["pk_anio"] = df_long["pk_anio"].astype(float)

# Filtrar para eliminar el año 1954
df_long = df_long[df_long["pk_anio"] != 1954]

# Crear la columna "Fecha" en formato '1-ene.', '2-ene.', etc.
df_long["Fecha"] = df_long["dia"].astype(str) + "-" + df_long["pk_mes"].apply(lambda x: nombres_meses[x-1])

df_long = df_long.groupby(["Fecha", "pk_anio"], as_index=False)["valor"].mean()

# Reorganizar el DataFrame para que los años sean columnas y las fechas sean el índice
df_pivot = df_long.pivot(index="Fecha", columns="pk_anio", values="valor")

# Asegurar que solo haya 365 filas y estén en orden correcto
fechas_ordenadas = [f"{d+1}-{m}" for m in nombres_meses for d in range(31) if not (m == "feb." and d >= 28)]  # Evita 29-feb.
df_pivot = df_pivot.reindex(fechas_ordenadas)

# Hacer que 'Fecha' sea el índice
df_pivot.index.name = "Fecha"

# Eliminar 'pk_anio' del encabezado
df_pivot.columns.name = None

# Mostrar el DataFrame transformado
df_pivot

,1978.0,1979.0,1980.0,1981.0,1982.0,1983.0,1984.0,1985.0,1987.0,1988.0,...,2000.0,2001.0,2002.0,2003.0,2004.0,2005.0,2006.0,2007.0,2008.0,2009.0
Fecha,,,,,,,,,,,,,,,,,,,,,
1-ene.,NaN,7.50162,4.71471,6.30406,7.08848,56.43795,7.83200,22.03484,54.21530,19.34821,...,5.10384,8.99800,3.27235,6.22605,4.99700,8.29300,3.50006,47.412,5.598,7.098
2-ene.,NaN,8.52518,4.55758,6.01118,7.06260,49.66361,7.82026,15.32345,53.58320,25.25915,...,4.96598,9.41844,3.08453,6.05659,4.99700,7.63103,3.43795,27.654,5.569,6.955
3-ene.,NaN,8.92056,4.58673,5.99087,7.05988,39.66199,7.66418,12.74813,52.90700,22.46642,...,5.06430,7.24100,2.85820,5.76182,4.99700,8.21358,3.45841,36.300,5.524,6.587
4-ene.,NaN,7.95500,4.73477,5.99988,7.05715,33.12414,7.56847,11.50462,52.80410,20.30216,...,4.93594,6.74492,2.63186,5.74318,4.80770,7.36356,3.15445,254.720,5.240,6.410
5-ene.,NaN,7.60956,4.74694,5.89662,7.04182,28.60423,7.66787,10.93037,52.64238,18.48249,...,4.75861,6.06611,2.40553,5.57219,4.68717,7.26995,2.84463,216.033,5.133,6.020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27-dic.,6.984,5.09068,6.71575,7.52653,412.44230,9.02788,14.86158,12.63838,15.46918,0.99513,...,4.04871,5.28375,6.69971,5.26944,8.89600,3.77552,15.60148,6.218,7.281,132.201
28-dic.,7.150,4.99976,6.64275,7.44998,235.03430,9.02100,13.74238,9.75723,15.12713,4.49455,...,6.11144,4.91093,6.94608,4.98799,8.89600,3.76878,13.24081,5.829,7.418,179.033
29-dic.,7.893,4.98500,6.59028,7.40981,133.59190,9.02100,13.10626,6.97683,18.41604,4.75991,...,7.49961,4.46244,6.67351,4.93916,8.89600,3.69542,13.55898,5.764,7.288,180.005


In [6]:
df_pivot.to_excel("C:/Users/moise/OneDrive/Documentos/UPY/project/Estancia ENES/Scraper_and_mapping/Databases/DD11075_transform.xlsx", index=True)